In [2]:
from IPython.display import display
import ipywidgets as widgets

# Create a file upload widget
upload_widget = widgets.FileUpload(
    accept='.txt',  # Accept only .txt files
    multiple=True    # Allow multiple file uploads
)

# Display the upload widget
display(upload_widget)



FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [3]:

import os
import re

def extract_models(file_content, delimiter='print model ok!'):
    """
    Splits the file content into individual models based on the delimiter.
    
    :param file_content: The raw content of the text file.
    :param delimiter: The string that separates models.
    :return: A list of model strings.
    """
    models = file_content.split(delimiter)
    # Remove any empty strings and strip whitespace
    models = [model.strip() for model in models if model.strip()]
    return models

def format_model_to_c_constants(model_text, base_name, model_index, array_type='int16_t', frame_num_type='uint16_t', numbers_per_line=12):
    """
    Formats a single model's text into C constants.
    
    :param model_text: The raw text of the model.
    :param base_name: The base name derived from the file name.
    :param model_index: The index of the model (e.g., 0, 1, 2, 3).
    :param array_type: Data type of the array (e.g., int16_t).
    :param frame_num_type: Data type of the frame number (e.g., uint16_t).
    :param numbers_per_line: Number of numbers per line in the array for readability.
    :return: Formatted C code for the model as a string.
    """
    # Extract the frame number using regex
    frame_num_match = re.search(r'frm_num\s*=\s*(\d+)', model_text)
    if not frame_num_match:
        raise ValueError("Frame number (frm_num=...) not found in the model.")
    frame_num = int(frame_num_match.group(1))
    
    # Extract all numbers after 'frm_num=...' up to the end of the model
    numbers_section = model_text[frame_num_match.end():]
    
    # Use regex to find all integers (including negative numbers)
    numbers = re.findall(r'[-]?\d+', numbers_section)
    numbers = [int(num) for num in numbers]
    
    # Define variable names
    frame_num_variable = f"fram_num_{base_name}_{model_index}"
    array_variable = f"{base_name}_{model_index}"
    
    # Start building the C code
    c_code = ""
    
    # Add frame number constant
    c_code += f"const {frame_num_type} {frame_num_variable} = {frame_num};\n\n"
    
    # Add array declaration
    c_code += f"const {array_type} {array_variable}[vv_frm_max*mfcc_num] = {{\n"
    
    # Add numbers with specified numbers per line
    for i in range(0, len(numbers), numbers_per_line):
        line_numbers = numbers[i:i+numbers_per_line]
        line = ', '.join(str(num) for num in line_numbers)
        if i + numbers_per_line >= len(numbers):
            c_code += f"    {line}\n"
        else:
            c_code += f"    {line},\n"
    
    c_code += "};\n\n"
    
    return c_code

def process_files(input_files, output_header='voice_model.h', models_per_file=4):
    """
    Processes multiple text files and generates a C header file with all voice models.
    
    :param input_files: List of paths to input text files.
    :param output_header: Path to the output C header file.
    :param models_per_file: Number of models expected per input file.
    """
    header = """#ifndef __VOICE_MODEL_H
#define __VOICE_MODEL_H

#include <stdint.h>
#include "util/MFCC.h"

#ifdef __cplusplus
extern "C" {
#endif

"""

    footer = """
#ifdef __cplusplus
}
#endif

#endif
"""

    all_c_constants = header

    for file_path in input_files:
        base_name = os.path.splitext(os.path.basename(file_path))[0]  # e.g., 'green' from 'green.txt'
        with open(file_path, 'r') as f:
            content = f.read()
        
        models = extract_models(content)
        
        if len(models) != models_per_file:
            print(f"Warning: Expected {models_per_file} models in '{file_path}', but found {len(models)}.")
        
        for index, model in enumerate(models):
            try:
                c_constants = format_model_to_c_constants(model, base_name, index)
                all_c_constants += c_constants
            except ValueError as e:
                print(f"Error processing model {index} in file '{file_path}': {e}")

    all_c_constants += footer

    # Write to the output header file
    with open(output_header, 'w') as f:
        f.write(all_c_constants)
    
    print(f"C header file '{output_header}' has been generated successfully.")


import io

uploaded_files = upload_widget.value

# Save uploaded files to the current directory
for filename, file_info in uploaded_files.items():
    with open(filename, 'wb') as f:
        f.write(file_info['content'])
    print(f"Saved uploaded file: {filename}")


# Automatically include all uploaded files
input_files = list(uploaded_files.keys())

# Specify the number of models per file
models_per_file = 6  # Adjust this number based on your files

# Specify the output header file name
output_header = 'voice_model.h'

# Process the files and generate the header
if input_files:
    process_files(input_files, output_header=output_header, models_per_file=models_per_file)
else:
    print("No input files found. Please upload text files using the upload widget.")


Saved uploaded file: jarvis_demo_mode.txt
Saved uploaded file: wake_up_jarvis.txt
Saved uploaded file: lights_off_jarvis.txt
Saved uploaded file: jarvis_lights_on.txt
Saved uploaded file: jarvis_goodmorning.txt
Saved uploaded file: goodnight_jarvis.txt
Saved uploaded file: cooking_mode.txt
Saved uploaded file: bye_bye_jarvis.txt
Saved uploaded file: alfred_batcave_on.txt
Saved uploaded file: clap_clap.txt
C header file 'voice_model.h' has been generated successfully.
